## Primeiros passos com Análises espaciais em Python

pacotes que vamos usar

* Geopandas - combina as capacidades de análises que o Pandas oferece com  os pacotes shapely e fiona - voltados para gerenciamento de dados espaciais.

**Observações:**
 > Diferença entre DataFrame e GeoDataFrame - no geodrataframe existe uma coluna com dados geométricos, que por padrão é chamada de "geometry". A mesma é uma GeoSeries que contêm dados geométricos (pontos, linhas, polígonos, multipolígonos, etc...) como objetos modelados.

### Shapefile

Conjunto de arquivos que possibilitam a existência de um mapa atrelado a base de dados. 

Existem 3 tipos de dados vetoriais:
>1° Pontos;

>2° Linhas;

>3° Polígonos.

Dados geoespaiciais no formato de vetor em geral são armazenados em shapefile. Como as estruturas de pontos, linhas e polígonos são diferentes, um shapefile só pode ter um tipo de vetor.

Um shapefile é criado por 3 ou mais arquivos **todos devem ter o mesmo nome e estarem no mesmo diretório (pasta)** para garantir que você consiga trabalhar com ele.

**Estrutura de um shapefile**

There are 3 key files associated with any and all shapefiles:

>.shp: the file that contains the geometry for all features.

>.shx: the file that indexes the geometry.

>.dbf: the file that stores feature attributes in a tabular format.

Sometimes, a shapefile will have other associated files including:

>.prj: the file that contains information on projection format including the coordinate system and projection information. It is a plain text file describing the projection using well-known text (WKT) format.

>.sbn and .sbx: the files that are a spatial index of the features.

>.shp.xml: the file that is the geospatial metadata in XML format, (e.g. ISO 19115 or XML format).

##### Aqui utilizaremos a biblioteca geopandas para trabalhar com dados vetorizados em Python e matplotlib.pyplot para plotar nossas visualizações.

In [1]:
#Importando as bibliotecas que vamos utilizar

from matplotlib import pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
#Biblioteca usada para acessar o diretório
from pathlib import Path

In [3]:
#Importando o arquivo que vamos usar

path = Path("Python/shapefile_sp/")
mundo_file = path/"mundo.shp"

In [5]:
sp_data = gpd.read_file(mundo_file)

In [6]:
sp_data.shape

Nosso conjunto de dados é composto por 3 colunas, sendo uma delas a **geometry** - coluna que apresenta os dados geométricos, nesse caso são polígonos.

Além dela temos as colunas:
- NM_MUNICIP -> nome dos municípios de SP 
- CD_GEOCMU -> códigos dos municípios

Em shapefiles cada linha da tabela representa um objeto espacial, no nosso caso cada linha é um município do estado de São Paulo.

In [ ]:
type(sp_data)

In [ ]:
sp_data.total_bounds

In [ ]:
#Verificando o CRS (sistema de coordenadas referenciais)
sp_data.crs

In [ ]:
sp_data.geometry.head()

In [ ]:
#Visualizando o shapefile usando o método plot()
fig, ax = plt.subplots(1,1, figsize=(30,15))
sp_data.plot(ax=ax, column='NM_MUNICIP');

##### Manipulando dados no shapefile

+ Vamos calcular a área de cada municípo utilizando a função GeoSeries.area que retorna uma serie contendo a área de cada município expressa em unidades de CRS(sistema de coordenada referencial)

+ Antes disso precisamos mudar nossa unidade do CRS de graus para para o sistema cartesiano que usa metros²



In [ ]:
#primeiro vamos fazer uma cópia dos arquivos para guardar
sp_data_original_data = sp_data.copy()

In [ ]:
#Agora vamos reescrever o CRS para o CRS desejado. Importante colocar o parâmetro allow_override=True para evitar 
#erros ao sobreescrever o CRS
sp_mun = sp_data.set_crs(epsg=6933, allow_override=True)

In [ ]:
#Verificando se o CRS mudou
sp_mun.crs

In [ ]:
#Com o nosso sistema de coordenadas referenciais modificado, é hora de calcular a área dos municípios
#Dividindo por 10⁶ porque .area retorna em metros².

sp_mun.area

In [ ]:
#Criando uma variável no shapefile para armazenar as áreas calculadas
sp_mun['Area'] = sp_mun.area/10e6

In [ ]:
sp_mun.head()

In [ ]:
#Unindo dois arquivos de shapefile 

#Importando o outro arquivos que vamos usar

path = Path("Python/shapefile_mundo/")
mundo_file = path/"mundo.shp"

In [ ]:
mundo_data = gpd.read_file(mundo_file,  encoding='latin1')

In [ ]:
mundo_data.head()

In [ ]:
#Unindo os dois dataframes (sp_mun e mundo_data)

data_unificado = gpd.GeoDataFrame(pd.concat([sp_mun, mundo_data]))

In [ ]:
data_unificado.head()

In [ ]:
#Salvando um shapefile 
output = "~/Python/municipios_sp_finalizado.shp"

sp_mun.to_file(output)

#### Trabalhando com outro shapefile


In [ ]:
#Importando o arquivo

path_file = "~/Python/shapefile_mundo/"

mundo_file = path/"mundo.shp"

In [ ]:
mundo_df = gpd.read_file(mundo_file)

In [ ]:
#plotando o objeto
fig, ax = plt.subplots(1,1, figsize=(15,15))
mundo_data.plot(ax=ax);

In [ ]:
mundo_data.head()

In [ ]:
mundo_data.shape

In [ ]:
#Selecionando apenas os paises da américa do sul 
south_america = mundo_data.loc[mundo_data['contnnt'] == 'South America']

In [ ]:
south_america.head()

In [ ]:
#Plotando os países da América do Sul
fig, ax = plt.subplots(1,1, figsize=(15,10))
south_america.plot(ax=ax);

In [ ]:
#Vamos criar agora uma combinação de shapefiles usando as bases de dados da pasta Mercosul

#Primeiro vamos importar os shapefiles
path = Path("~/Python/shapefile_mercosul/")

brasil = path/"brasil_shapefile.shp"
argentina = path/"argentina_shapefile.shp"
paraguai = path/"paraguai_shapefile.shp"
venezuela = path/"venezuela_shapefile.shp"

In [ ]:
#Carregando os arquivos dentro de um geodataframe
brasil_df = gpd.read_file(brasil)
argentina_df = gpd.read_file(argentina)
paraguai_df = gpd.read_file(paraguai)
venezuela_df = gpd.read_file(venezuela)

In [ ]:
brasil_df.shape, argentina_df.shape

In [ ]:
paraguai_df.shape, venezuela_df.shape

In [ ]:
venezuela_df.head()

In [ ]:
#Concatenando os geodataframes
mercosul_df = pd.concat([brasil_df,  argentina_df, paraguai_df, venezuela_df])

In [ ]:
mercosul_df.shape

In [ ]:
mercosul_df.head()

In [ ]:
#Visualziando os dados
fig,ax = plt.subplots(1,1, figsize=(15,15))
mercosul_df.plot(ax=ax);

# Referências

* Parte inicial de dados geoespaciais estou usando esse tutorial

> https://sustainability-gis.readthedocs.io/en/latest/lessons/L1/intro-to-python-geostack.html

* Tutorias para trabalhar com Shapefile

> Início bem importante para pegar a teoria inicial - guillaumedueymes.com/post/shapefiles_in_python/



>Esse aqui tem vários exemplos de plotagem - https://www.earthdatascience.org/workshops/gis-open-source-python/intro-vector-data-python/

> Tutorial **Intro to Python GIS** - https://automating-gis-processes.github.io/CSC/notebooks/L2/projections.html